### Homework Week 1

In [2]:
# imports
from datetime import date

import pandas_datareader as pdr
import yfinance as yf

/Users/evgeniy/stock-markets-analytics-zoomcamp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [15]:
# get GDPC1 data starting 2022
start = date(2022, 1, 1)
gdp_df = pdr.DataReader("GDPC1", "fred", start=start)

# calculate GDPC1_yoy as % and get mean for the last 4 quarters
gdp_df['GDPC1_yoy'] = (gdp_df.GDPC1 / gdp_df.GDPC1.shift(4) - 1) * 100
gdp_df.loc['2023-01-01':]['GDPC1_yoy'].mean().round(1)

2.5

In [16]:
# get DGS2 and DGS10
start = date(2000, 1, 1)
dsg2 = pdr.DataReader("DGS2", "fred", start=start)
dsg10 = pdr.DataReader("DGS10", "fred", start=start)

# join and calculate Inverse Treasury Yield
dsg2_10 = dsg2.join(dsg10)
dsg2_10['DGS10-DGS2'] = dsg2_10['DGS10'] - dsg2_10['DGS2']
dsg2_10['DGS10-DGS2'].min().round(1)

-1.1

In [17]:
# get indexes from Yahoo
indexes = yf.download(
    tickers = ["^GSPC", "^MXX"],
    start = "2019-04-09",
    end = "2024-04-09",
    interval = "1d",
    progress=False,
    )
indexes = indexes['Close']

# calc growth rate % and round it
indexes['snp_growth_%'] = ((indexes['^GSPC'] / indexes.iloc[0]['^GSPC'] - 1) * 100).round()
indexes['ipc_growth_%'] = ((indexes['^MXX'] / indexes.iloc[0]['^MXX'] - 1) * 100).round()
indexes.tail()

Ticker,^GSPC,^MXX,snp_growth_%,ipc_growth_%
Date,,,,
2024-04-02,5205.810059,57581.808594,81.0,28.0
2024-04-03,5211.490234,57503.390625,81.0,27.0
2024-04-04,5147.209961,57882.761719,79.0,28.0
2024-04-05,5204.339844,58092.441406,81.0,29.0
2024-04-08,5202.390137,57989.941406,81.0,28.0


In [18]:
tickers = ['2222.SR', 'BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']
stocks = yf.download(
    tickers = tickers,
    start = "2023-01-01",
    end = "2023-12-31",
    interval = "1d",
    progress=False,
    )
stocks = stocks['Adj Close'].agg(['min', 'max']).transpose()
stocks['ratio'] = ((stocks['max'] - stocks['min']) / stocks['max']).round(2)
stocks['ratio'].max()

0.42

In [3]:
tickers = ['2222.SR', 'BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']
start = '2023-01-01'
end = '2023-12-31'
max_yield = 0

for ticker in tickers:
    actions = yf.Ticker(ticker).get_actions()
    sum_dividends = actions.loc[start:end]['Dividends'].sum()
    stock = yf.download(
        tickers = ticker,
        start = start,
        end = end,
        interval = "1d",
        progress=False,
        )
    price_max =  stock.iloc[-1]['Adj Close']
    div_yield =  (sum_dividends / price_max * 100).round(1)
    print(f'{ticker}: {div_yield}')
    if  div_yield > max_yield:
        max_yield = div_yield
print(f"Max: {max_yield}")

2222.SR: 2.8
BRK-B: 0.0
AAPL: 0.5
MSFT: 0.7
GOOG: 0.0
JPM: 2.4
Max: 2.8
